In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
planets = sns.load_dataset('planets')
print(planets.shape)
planets.head()

(1035, 6)


,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


# basic aggregations

In [4]:
planets.mean()

number               1.785507
orbital_period    2002.917596
mass                 2.638161
distance           264.069282
year              2009.070531
dtype: float64

In [15]:
planets.aggregate(['mean','std','mad','median'])

,method,number,orbital_period,mass,distance,year
mad,NaN,0.903144,3181.747643,2.580056,318.100885,3.056798
mean,NaN,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,NaN,1.240976,26014.728304,3.818617,733.116493,3.972567
median,NaN,1.000000,39.979500,1.260000,55.250000,2010.000000


In [12]:
planets.aggregate({'mass':'mean','year':'median'})

mass       2.638161
year    2010.000000
dtype: float64

In [14]:
planets.aggregate({'mass':['mean','std'],'year':['min','median']})

,mass,year
mean,2.638161,NaN
median,NaN,2010.0
min,NaN,1989.0
std,3.818617,NaN


In [16]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


# Group by

In [17]:
planets.head(5)

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [62]:
planets.groupby('method')

In [55]:
planets.groupby('method')['number']

In [102]:
planets.groupby('method').size()

method
Astrometry                         2
Eclipse Timing Variations          9
Imaging                           38
Microlensing                      23
Orbital Brightness Modulation      3
Pulsar Timing                      5
Pulsation Timing Variations        1
Radial Velocity                  553
Transit                          397
Transit Timing Variations          4
dtype: int64

In [105]:
planets.groupby('method')['year'].median()

method
Astrometry                       2011.5
Eclipse Timing Variations        2010.0
Imaging                          2009.0
Microlensing                     2010.0
Orbital Brightness Modulation    2011.0
Pulsar Timing                    1994.0
Pulsation Timing Variations      2007.0
Radial Velocity                  2009.0
Transit                          2012.0
Transit Timing Variations        2012.5
Name: year, dtype: float64

In [107]:
planets.groupby('method').aggregate(['median','count']).stack().stack().unstack(level=[1]).head(7)

median  count
method                                                   
Astrometry                number             1.000    2.0
                          orbital_period   631.180    2.0
                          mass                 NaN    0.0
                          distance          17.875    2.0
                          year            2011.500    2.0
Eclipse Timing Variations number             2.000    9.0
                          orbital_period  4343.500    9.0

## Iterate group by object

In [63]:
for (method,group) in planets.groupby('method'):
    print(method,group.shape)

Astrometry (2, 6)
Eclipse Timing Variations (9, 6)
Imaging (38, 6)
Microlensing (23, 6)
Orbital Brightness Modulation (3, 6)
Pulsar Timing (5, 6)
Pulsation Timing Variations (1, 6)
Radial Velocity (553, 6)
Transit (397, 6)
Transit Timing Variations (4, 6)


## Filter groupby

In [87]:
planets['morerotationtime']=planets['orbital_period']>365
planets.head()
planets.groupby('morerotationtime').median()
planets=planets.drop('morerotationtime',axis=1)

In [88]:
planets.groupby('method').median()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,1.0,631.180000,NaN,17.875,2011.5
Eclipse Timing Variations,2.0,4343.500000,5.125,315.360,2010.0
Imaging,1.0,27500.000000,NaN,40.395,2009.0
Microlensing,1.0,3300.000000,NaN,3840.000,2010.0
Orbital Brightness Modulation,2.0,0.342887,NaN,1180.000,2011.0
Pulsar Timing,3.0,66.541900,NaN,1200.000,1994.0
Pulsation Timing Variations,1.0,1170.000000,NaN,NaN,2007.0
Radial Velocity,1.0,360.200000,1.260,40.445,2009.0
Transit,1.0,5.714932,1.470,341.000,2012.0


In [89]:
def filter_func(x):
    return x['year'].median() >2010
#removes everything from groups that do not match the filter_func.
#removes entry from data frame too
planets.groupby('method').filter(filter_func).head()

,method,number,orbital_period,mass,distance,year
91,Transit,1,1.508956,NaN,NaN,2008
92,Transit,1,1.742994,NaN,200.0,2008
93,Transit,1,4.256800,NaN,680.0,2008
94,Transit,1,9.202050,NaN,NaN,2008
95,Transit,1,4.037896,NaN,NaN,2009


In [90]:
planets.groupby('method').filter(filter_func).groupby('method').median()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,1.0,631.180000,NaN,17.875,2011.5
Orbital Brightness Modulation,2.0,0.342887,NaN,1180.000,2011.0
Transit,1.0,5.714932,1.47,341.000,2012.0
Transit Timing Variations,2.0,57.011000,NaN,855.000,2012.5


## Transform

good for operations within same column

example:substract column from its groupwise mean

In [91]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [92]:
planets.groupby('method').transform(lambda x : x-x.mean()).head()

,number,orbital_period,mass,distance,year
0,-0.721519,-554.05468,4.469301,25.799792,-1.518987
1,-0.721519,51.41932,-0.420699,5.349792,0.481013
2,-0.721519,-60.35468,-0.030699,-31.760208,3.481013
3,-0.721519,-497.32468,16.769301,59.019792,-0.518987
4,-0.721519,-307.13468,7.869301,67.869792,1.481013


## Apply groupby

Apply own function in groupby. Good for multi column operations.

example: divide a column with group mean of another coumn. + create a new column

In [93]:
def norm_mass_by_dist(x):
    x['mass/meandist']=x['mass']/x['distance'].mean()
    return x
planets.groupby('method').apply(norm_mass_by_dist).head()

,method,number,orbital_period,mass,distance,year,mass/meandist
0,Radial Velocity,1,269.300,7.10,77.40,2006,0.137596
1,Radial Velocity,1,874.774,2.21,56.95,2008,0.042829
2,Radial Velocity,1,763.000,2.60,19.84,2011,0.050387
3,Radial Velocity,1,326.030,19.40,110.62,2007,0.375967
4,Radial Velocity,1,516.220,10.50,119.47,2009,0.203488


## Split key

### Ways to do this
1. Pass the column name
2. By passing group key array [0,1,1,0]. e.g. pass the whole column
3. Dictionary mapping of index to group key
3. Through split function str.lower, split_key. 
    Takes input as index value(**set_index('orbital_period')**) and outputs the group

In [109]:
def split_key(x):
    if x<100:
        return '<100'
    elif x<365:
        return '<365'
    else:
        return '>365'
planets.set_index('orbital_period').groupby(split_key).aggregate(['count','mean'])

number           orbital_period               mass           distance  \
      count      mean          count         mean count      mean    count   
<100    100  1.620000             87  5212.829836    53  4.151683       84   
<365    265  1.807547            263   561.767047   199  2.263460      261   
>365    670  1.801493            642  2158.307362   261  2.616509      463   

                  year               
            mean count         mean  
<100   77.520952   100  2007.500000  
<365  124.227701   265  2008.200000  
>365  376.744687   670  2009.649254

In [115]:
planets.set_index('orbital_period').groupby(['method',split_key])['number'].sum().unstack().fillna(0)

,<100,<365,>365
method,,,
Astrometry,0.0,1.0,1.0
Eclipse Timing Variations,0.0,0.0,15.0
Imaging,0.0,0.0,50.0
Microlensing,0.0,0.0,27.0
Orbital Brightness Modulation,5.0,0.0,0.0
Pulsar Timing,10.0,0.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0
Radial Velocity,377.0,171.0,404.0
Transit,719.0,57.0,0.0
